In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

# the usuals
import geopandas as gpd
import matplotlib.pyplot as plt

import contextily as ctx
from sodapy import Socrata


# to explore point patterns
from pointpats import centrography
from matplotlib.patches import Ellipse

In [2]:
# to import data from LA Data portal
from sodapy import Socrata
import pandas as pd
import geopandas as gpd

# connect to the data portal
client = Socrata("data.lacounty.gov", None)

results = client.get("9trm-uz8i",
                    limit=50000,
                    where="taxratearea_city='INGLEWOOD' AND rollyear=2006 AND effectiveyearbuilt=2006")

df6 = pd.DataFrame.from_records(results)

In [3]:
df6.center_lon = df6.center_lon.astype('float')
df6.center_lat = df6.center_lat.astype('float')

In [5]:
df6.drop(df6[df6.center_lon==0].index,inplace=True)

In [6]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176 entries, 0 to 175
Data columns (total 51 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   situszip                   176 non-null    object 
 1   taxratearea_city           176 non-null    object 
 2   ain                        176 non-null    object 
 3   rollyear                   176 non-null    object 
 4   taxratearea                176 non-null    object 
 5   assessorid                 176 non-null    object 
 6   propertylocation           175 non-null    object 
 7   usetype                    176 non-null    object 
 8   usecode                    176 non-null    object 
 9   usecodedescchar1           176 non-null    object 
 10  usecodedescchar2           176 non-null    object 
 11  usecodedescchar3           176 non-null    object 
 12  usecodedescchar4           176 non-null    object 
 13  totbuildingdatalines       176 non-null    object 

In [7]:
df6.head(5)

,situszip,taxratearea_city,ain,rollyear,taxratearea,assessorid,propertylocation,usetype,usecode,usecodedescchar1,...,situsfraction,situsdirection,situsstreet,situsunit,situscity,situszip5,rowid,center_lat,center_lon,location_1
0,90003,INGLEWOOD,4020021012,2006,04571,4020-021-012,219 W FLORENCE AVE INGLEWOOD CA 90003,C/I,3100,Industrial,...,,W,FLORENCE AVE,,INGLEWOOD CA,90003,20064020021012,33.965664,-118.358584,"{'latitude': '33.96566374', 'longitude': '-118..."
1,90305-1885,INGLEWOOD,4025021008,2006,04596,4025-021-008,3606 W LUTHER LN INGLEWOOD CA 90305,SFR,0100,Residential,...,,W,LUTHER LN,,INGLEWOOD CA,90305,20064025021008,33.953661,-118.332928,"{'latitude': '33.95366103', 'longitude': '-118..."
2,90305-1999,INGLEWOOD,4025023034,2006,04596,4025-023-034,3530 W CHAUCER LN INGLEWOOD CA 90305,SFR,0100,Residential,...,,W,CHAUCER LN,,INGLEWOOD CA,90305,20064025023034,33.953446,-118.331317,"{'latitude': '33.95344623', 'longitude': '-118..."
3,90305-1885,INGLEWOOD,4025021010,2006,04596,4025-021-010,3610 W LUTHER LN INGLEWOOD CA 90305,SFR,0100,Residential,...,,W,LUTHER LN,,INGLEWOOD CA,90305,20064025021010,33.953902,-118.332928,"{'latitude': '33.95390245', 'longitude': '-118..."
4,90305-1885,INGLEWOOD,4025021009,2006,04596,4025-021-009,3608 W LUTHER LN INGLEWOOD CA 90305,SFR,0100,Residential,...,,W,LUTHER LN,,INGLEWOOD CA,90305,20064025021009,33.953776,-118.332928,"{'latitude': '33.95377608', 'longitude': '-118..."


In [8]:
df6 = df6[['taxratearea_city','rollyear','propertylocation','usecodedescchar1','effectiveyearbuilt','roll_totalvalue','center_lat','center_lon']]

In [9]:
df6.sample()

,taxratearea_city,rollyear,propertylocation,usecodedescchar1,effectiveyearbuilt,roll_totalvalue,center_lat,center_lon
6,INGLEWOOD,2006,3616 W LUTHER LN INGLEWOOD CA 90305,Residential,2006,93490,33.953661,-118.333126


In [10]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176 entries, 0 to 175
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   taxratearea_city    176 non-null    object 
 1   rollyear            176 non-null    object 
 2   propertylocation    175 non-null    object 
 3   usecodedescchar1    176 non-null    object 
 4   effectiveyearbuilt  176 non-null    object 
 5   roll_totalvalue     176 non-null    object 
 6   center_lat          176 non-null    float64
 7   center_lon          176 non-null    float64
dtypes: float64(2), object(6)
memory usage: 12.4+ KB


In [11]:
df6['roll_totalvalue'].astype('int')

0      330494
1       93490
2      183490
3       93490
4       93490
        ...  
171     93490
172     93490
173    436890
174     93490
175     93490
Name: roll_totalvalue, Length: 176, dtype: int64

In [12]:
df6.dtypes


taxratearea_city       object
rollyear               object
propertylocation       object
usecodedescchar1       object
effectiveyearbuilt     object
roll_totalvalue        object
center_lat            float64
center_lon            float64
dtype: object

In [13]:
df6["roll_totalvalue"] = df6['roll_totalvalue'].astype('int')
df6.dtypes

taxratearea_city       object
rollyear               object
propertylocation       object
usecodedescchar1       object
effectiveyearbuilt     object
roll_totalvalue         int64
center_lat            float64
center_lon            float64
dtype: object

In [14]:
df6

,taxratearea_city,rollyear,propertylocation,usecodedescchar1,effectiveyearbuilt,roll_totalvalue,center_lat,center_lon
0,INGLEWOOD,2006,219 W FLORENCE AVE INGLEWOOD CA 90003,Industrial,2006,330494,33.965664,-118.358584
1,INGLEWOOD,2006,3606 W LUTHER LN INGLEWOOD CA 90305,Residential,2006,93490,33.953661,-118.332928
2,INGLEWOOD,2006,3530 W CHAUCER LN INGLEWOOD CA 90305,Residential,2006,183490,33.953446,-118.331317
3,INGLEWOOD,2006,3610 W LUTHER LN INGLEWOOD CA 90305,Residential,2006,93490,33.953902,-118.332928
4,INGLEWOOD,2006,3608 W LUTHER LN INGLEWOOD CA 90305,Residential,2006,93490,33.953776,-118.332928
...,...,...,...,...,...,...,...,...
171,INGLEWOOD,2006,9115 S CULLEN WY INGLEWOOD CA 90305,Residential,2006,93490,33.954287,-118.335093
172,INGLEWOOD,2006,3605 W LUTHER LN INGLEWOOD CA 90305,Residential,2006,93490,33.954320,-118.332683
173,INGLEWOOD,2006,3609 W LUTHER LN INGLEWOOD CA 90305,Residential,2006,436890,33.954319,-118.332801
174,INGLEWOOD,2006,9111 S CULLEN WY INGLEWOOD CA 90305,Residential,2006,93490,33.954445,-118.335093


### OK SO FAR I HAVE EXTRACTED DATA FROM THE ASSESSORS DATA PORTAL REGARDING WHATEVER THEY HAVE THAT CAME INTO CONSTRUCTION EFFECTIVE OF 2006. THATS IT, JUST THAT SINGLE YEAR. WHY? BECAUSE THATS THE YEAR IT STARTS AND IT WOULD SERVE TO SEE WHAT CHANGES ARE NOTICEABLE FROM THEN UNTIL THE LATEST UPDATE OF 2020. 

In [15]:
df6.drop(df6[df6.roll_totalvalue==0].index,inplace=True)

In [16]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176 entries, 0 to 175
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   taxratearea_city    176 non-null    object 
 1   rollyear            176 non-null    object 
 2   propertylocation    175 non-null    object 
 3   usecodedescchar1    176 non-null    object 
 4   effectiveyearbuilt  176 non-null    object 
 5   roll_totalvalue     176 non-null    int64  
 6   center_lat          176 non-null    float64
 7   center_lon          176 non-null    float64
dtypes: float64(2), int64(1), object(5)
memory usage: 16.4+ KB


In [17]:
df6.sort_values(by="roll_totalvalue")

,taxratearea_city,rollyear,propertylocation,usecodedescchar1,effectiveyearbuilt,roll_totalvalue,center_lat,center_lon
175,INGLEWOOD,2006,9107 S CULLEN WY INGLEWOOD CA 90305,Residential,2006,93490,33.954603,-118.335094
100,INGLEWOOD,2006,3618 W SCRIBNER LN INGLEWOOD CA 90305,Residential,2006,93490,33.954530,-118.332957
99,INGLEWOOD,2006,8007 S CULLEN WY INGLEWOOD CA 90305,Residential,2006,93490,33.955973,-118.334968
147,INGLEWOOD,2006,3645 W LUTHER LN INGLEWOOD CA 90305,Residential,2006,93490,33.954142,-118.333457
95,INGLEWOOD,2006,9039 S CULLEN WY INGLEWOOD CA 90305,Residential,2006,93490,33.955436,-118.335122
...,...,...,...,...,...,...,...,...
7,INGLEWOOD,2006,3628 W LUTHER LN INGLEWOOD CA 90305,Residential,2006,501000,33.953661,-118.333521
15,INGLEWOOD,2006,3626 W LUTHER LN INGLEWOOD CA 90305,Residential,2006,507500,33.953776,-118.333522
5,INGLEWOOD,2006,3630 W LUTHER LN INGLEWOOD CA 90305,Residential,2006,507500,33.953661,-118.333719
13,INGLEWOOD,2006,3634 W LUTHER LN INGLEWOOD CA 90305,Residential,2006,547000,33.953902,-118.333720


### OK NOW WE HAVE OUR DATA SORTED BY VALUE FROM LOWEST TO LARGEST. LETS DO THE SAME THING NOW BUT FOR 2020.

In [18]:
# connecting to the data portal
client = Socrata("data.lacounty.gov", None)

results = client.get("9trm-uz8i",
                    limit=50000,
                    where="taxratearea_city='INGLEWOOD' AND rollyear=2020 AND effectiveyearbuilt BETWEEN 2007 AND 2020")

df20 = pd.DataFrame.from_records(results)

In [19]:
df20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 51 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   situszip                   209 non-null    object
 1   taxratearea_city           210 non-null    object
 2   ain                        210 non-null    object
 3   rollyear                   210 non-null    object
 4   taxratearea                210 non-null    object
 5   assessorid                 210 non-null    object
 6   propertylocation           209 non-null    object
 7   usetype                    209 non-null    object
 8   usecode                    210 non-null    object
 9   usecodedescchar1           210 non-null    object
 10  usecodedescchar2           210 non-null    object
 11  usecodedescchar4           89 non-null     object
 12  totbuildingdatalines       210 non-null    object
 13  yearbuilt                  210 non-null    object
 14  effectivey

In [20]:
df20.head()

,situszip,taxratearea_city,ain,rollyear,taxratearea,assessorid,propertylocation,usetype,usecode,usecodedescchar1,...,situszip5,rowid,center_lat,center_lon,location_1,situsdirection,usecodedescchar3,parcelclassification,situsunit,situsfraction
0,90302-1140,INGLEWOOD,4002015038,2020,04569,4002-015-038,1300 CENTINELA AVE INGLEWOOD CA 90302,C/I,1100,Commercial,...,90302,20204002015038,33.976895079999998,-118.36175846,"{'latitude': '33.97689508', 'longitude': '-118...",NaN,NaN,NaN,NaN,NaN
1,90302-1131,INGLEWOOD,4002009011,2020,04569,4002-009-011,237 W 64TH PL INGLEWOOD CA 90302,SFR,0100,Residential,...,90302,20204002009011,33.980607040000002,-118.36196714,"{'latitude': '33.98060704', 'longitude': '-118...",W,NaN,NaN,NaN,NaN
2,90302-1341,INGLEWOOD,4002029024,2020,04569,4002-029-024,212 E 64TH PL INGLEWOOD CA 90302,R-I,0200,Residential,...,90302,20204002029024,33.980458939999998,-118.35632837999999,"{'latitude': '33.98045894', 'longitude': '-118...",E,4 Stories or Less,NaN,NaN,NaN
3,90302-1027,INGLEWOOD,4001020033,2020,04569,4001-020-033,1740 CENTINELA AVE INGLEWOOD CA 90302,C/I,2120,Commercial,...,90302,20204001020033,33.977548589999998,-118.36960769,"{'latitude': '33.97754859', 'longitude': '-118...",NaN,Fast Food - Auto Oriented,NaN,NaN,NaN
4,90305-1035,INGLEWOOD,4009029021,2020,04569,4009-029-021,2917 W 79TH ST INGLEWOOD CA 90305,SFR,0100,Residential,...,90305,20204009029021,33.967540030000002,-118.32498652,"{'latitude': '33.96754003', 'longitude': '-118...",W,NaN,NaN,NaN,NaN


In [21]:
df20 = df20[['taxratearea_city','rollyear','propertylocation','usecodedescchar1','effectiveyearbuilt','roll_totalvalue','center_lat','center_lon']]

In [22]:
df20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   taxratearea_city    210 non-null    object
 1   rollyear            210 non-null    object
 2   propertylocation    209 non-null    object
 3   usecodedescchar1    210 non-null    object
 4   effectiveyearbuilt  210 non-null    object
 5   roll_totalvalue     210 non-null    object
 6   center_lat          210 non-null    object
 7   center_lon          210 non-null    object
dtypes: object(8)
memory usage: 13.2+ KB


In [23]:
df20['roll_totalvalue'].astype('int')

0       2191923
1        719815
2        334664
3       1837922
4        535170
         ...   
205     4522329
206     1082921
207    17600000
208     3409646
209     1083534
Name: roll_totalvalue, Length: 210, dtype: int64

In [24]:
df20.dtypes


taxratearea_city      object
rollyear              object
propertylocation      object
usecodedescchar1      object
effectiveyearbuilt    object
roll_totalvalue       object
center_lat            object
center_lon            object
dtype: object

In [25]:
df20["roll_totalvalue"] = df20['roll_totalvalue'].astype('int')
df20.dtypes

taxratearea_city      object
rollyear              object
propertylocation      object
usecodedescchar1      object
effectiveyearbuilt    object
roll_totalvalue        int64
center_lat            object
center_lon            object
dtype: object

In [26]:
df20.center_lon = df20.center_lon.astype('float')
df20.center_lat = df20.center_lat.astype('float')

In [27]:
df20.dtypes

taxratearea_city       object
rollyear               object
propertylocation       object
usecodedescchar1       object
effectiveyearbuilt     object
roll_totalvalue         int64
center_lat            float64
center_lon            float64
dtype: object

In [28]:
df20.drop(df20[df20.center_lon==0].index,inplace=True)

In [29]:
df20.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210 entries, 0 to 209
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   taxratearea_city    210 non-null    object 
 1   rollyear            210 non-null    object 
 2   propertylocation    209 non-null    object 
 3   usecodedescchar1    210 non-null    object 
 4   effectiveyearbuilt  210 non-null    object 
 5   roll_totalvalue     210 non-null    int64  
 6   center_lat          210 non-null    float64
 7   center_lon          210 non-null    float64
dtypes: float64(2), int64(1), object(5)
memory usage: 14.8+ KB


In [30]:
df20.sort_values(by="effectiveyearbuilt")

,taxratearea_city,rollyear,propertylocation,usecodedescchar1,effectiveyearbuilt,roll_totalvalue,center_lat,center_lon
0,INGLEWOOD,2020,1300 CENTINELA AVE INGLEWOOD CA 90302,Commercial,2007,2191923,33.976895,-118.361758
101,INGLEWOOD,2020,1123 MAPLE ST INGLEWOOD CA 90301,Residential,2007,326480,33.948023,-118.354106
99,INGLEWOOD,2020,3629 W 107TH ST INGLEWOOD CA 90303,Residential,2007,989400,33.939068,-118.336226
98,INGLEWOOD,2020,"3720 W 107TH STREET, B INGLEWOOD CA 90303",Residential,2007,298605,33.938454,-118.338186
92,INGLEWOOD,2020,3736 W 107TH ST INGLEWOOD CA 90303,Residential,2007,398668,33.938457,-118.338738
...,...,...,...,...,...,...,...,...
109,INGLEWOOD,2020,3656 W 108TH ST INGLEWOOD CA 90303,Residential,2019,566511,33.937450,-118.336869
23,INGLEWOOD,2020,224 E HAZEL ST INGLEWOOD CA 90302,(missing),2019,744590,33.970229,-118.353287
94,INGLEWOOD,2020,3707 W 106TH ST INGLEWOOD CA 90303,Residential,2019,935000,33.940084,-118.337895
7,INGLEWOOD,2020,844 GLENWAY DR INGLEWOOD CA 90302,Residential,2019,918000,33.970811,-118.367912


In [ ]:
df20.index.is_unique

True

In [ ]:
df20.columns.is_unique

True

In [37]:
df20.index.duplicated()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [39]:
# drop all rows with NaN values
df20.dropna(axis=0,inplace=True)

In [40]:
df20.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209 entries, 0 to 209
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   taxratearea_city    209 non-null    object 
 1   rollyear            209 non-null    object 
 2   propertylocation    209 non-null    object 
 3   usecodedescchar1    209 non-null    object 
 4   effectiveyearbuilt  209 non-null    object 
 5   roll_totalvalue     209 non-null    int64  
 6   center_lat          209 non-null    float64
 7   center_lon          209 non-null    float64
dtypes: float64(2), int64(1), object(5)
memory usage: 14.7+ KB


In [43]:
df6.to_csv(r'Final Project 2006 data.csv', index = False)

In [44]:
df20.to_csv(r'Final Project After06 data.csv', index = False)